In [1]:
from __future__ import print_function

import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import label_binarize

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [31]:
def neural_net(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
#     layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
#     layer_2 = tf.nn.relu(layer_2)
    
    out_layer = tf.matmul(layer_1, weights['output']) + biases['output']
    return out_layer

In [6]:
def split_data(file):
    filename = [file]
    record_defaults = [tf.float32] * 75   # Eight required float columns
    dataset = tf.contrib.data.CsvDataset(filename, record_defaults)
    return dataset

In [15]:
train_df = pd.read_csv("Data/formated.csv")
test_df = pd.read_csv("Data/formatedTesting.csv")

train_df.columns = [col.strip() for col in train_df.columns]
test_df.columns = [col.strip() for col in test_df.columns]

# Delete duplicate columns
del train_df["e1"]
del train_df["e1.1"]
del test_df["e1"]
del test_df["e1.1"]

# Split the dataframe into features and labels arrays
train_features_array = train_df.iloc[:, :-1].values
train_labels_array = train_df.loc[:, "target Feature"].values
train_labels_binarized = label_binarize(train_labels_array, classes=[0,1,2])
print(train_labels_binarized)
test_features_array = test_df.iloc[:, :-1].values
test_labels_array = test_df.loc[:, "target Feature"].values
test_labels_binarized = label_binarize(test_labels_array, classes=[0,1,2])

[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [0 1 0]
 [1 0 0]
 [0 1 0]]


In [27]:
input_nodes = 72 # maybe 75 for a bias
output_nodes = 3
hLayers = 2
hLayerOne = 36 # half of input nodes
hLayerTwo = 36 # for sigmoid function

learning_rate = 0.01
#epochs = 1 # number of times to iterate over data
batches = 3
train_batch = int(len(train_df)/batches)
test_batch = int(len(test_df)/batches)
keep_prob = tf.placeholder("float")
print(keep_prob)
X = tf.placeholder("float", [None, input_nodes])
Y = tf.placeholder("float", [None, output_nodes])
print(X)
print(Y)

Tensor("Placeholder_6:0", dtype=float32)
Tensor("Placeholder_7:0", shape=(?, 72), dtype=float32)
Tensor("Placeholder_8:0", shape=(?, 3), dtype=float32)


In [28]:
weights = {
    'h1': tf.Variable(tf.random_normal([input_nodes, hLayerOne])),
#     'h2': tf.Variable(tf.random_normal([hLayerOne, hLayerTwo])),
    'output': tf.Variable(tf.random_normal([hLayerOne, output_nodes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([hLayerOne])),
#     'b2': tf.Variable(tf.random_normal([hLayerTwo])),
    'output': tf.Variable(tf.random_normal([output_nodes]))
}

In [29]:
# Construct model
logits = neural_net(X)

# Finding the cost of the algorithm
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

# Finding optimization of the algorithm
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
total_batch = train_features_array.shape[0]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for batch in range(1):
        avg_cost = 0.0
        x_batches = np.array_split(train_features_array, total_batch)
        y_batches = np.array_split(train_labels_binarized, total_batch)
        for i in range(6807):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                X: batch_x, 
                                Y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
            if i % 1000 == 0:
                print("Epoch:", '%04d' % (i+1), "cost=", "{:.9f}".format(avg_cost))
                
        print("Optimization Finished!")
        correct_prediction = tf.equal(tf.argmax(X, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
#         print("Accuracy:", accuracy.eval({X: test_features_array, Y: test_labels_binarized, keep_prob: 1.0}))

In [33]:
total_batch = train_features_array.shape[0]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(1, 101):
        avg_cost = 0.0
        _, c = sess.run([optimizer, cost], 
                        feed_dict={
                            X: train_features_array, 
                            Y: train_labels_binarized, 
                        })
        avg_cost += c / total_batch
        if i % 10 == 0:
            print("Epoch:", '%04d' % (i), "cost=", "{:.9f}".format(avg_cost))
                
    print("Optimization Finished!")            
    
    print()
    
    train_pred = tf.nn.softmax(logits)
    correct_train_prediction = tf.equal(tf.argmax(train_pred, 1), tf.argmax(Y, 1))
    train_accuracy = tf.reduce_mean(tf.cast(correct_train_prediction, "float"))
    print("Training accuracy: ", train_accuracy.eval({X: train_features_array, Y: train_labels_binarized}))

    test_pred = tf.nn.softmax(logits)
    correct_test_prediction = tf.equal(tf.argmax(test_pred, 1), tf.argmax(Y, 1))
    test_accuracy = tf.reduce_mean(tf.cast(correct_test_prediction, "float"))
    print("Test accuracy: ", test_accuracy.eval({X: test_features_array, Y: test_labels_binarized}))

Epoch: 0010 cost= 0.008139166
Epoch: 0020 cost= 0.005359549
Epoch: 0030 cost= 0.003164174
Epoch: 0040 cost= 0.001947266
Epoch: 0050 cost= 0.001628850
Epoch: 0060 cost= 0.001376411
Epoch: 0070 cost= 0.001207770
Epoch: 0080 cost= 0.001046849
Epoch: 0090 cost= 0.000908889
Epoch: 0100 cost= 0.000779213
Optimization Finished!

Training accuracy:  0.6591744
Test accuracy:  0.62902516
